https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata?synchronous=true

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
headers = {"Content-Type":"application/json; charset=utf-8"
           ,"x-powerbi-resourcekey":"4b92a22d-c5c1-4980-aa08-3f219122e6aa" }

payload = {"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"_","Entity":"_measures","Type":0},{"Name":"r","Entity":"RollingAvgs","Type":0},{"Name":"d","Entity":"Dates","Type":0},{"Name":"_1","Entity":"_titles","Type":0},{"Name":"p","Entity":"Param - MWD","Type":0}],"Select":[{"Measure":{"Expression":{"SourceRef":{"Source":"_"}},"Property":"n"},"Name":"_measures.n","NativeReferenceName":"Case Count"},{"Measure":{"Expression":{"SourceRef":{"Source":"_"}},"Property":"epi_curve_note"},"Name":"_measures.epi_curve_note","NativeReferenceName":"Note"},{"Measure":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"rollingavg_table"},"Name":"RollingAvgs.rollingavg_table","NativeReferenceName":"7-Day Average"},{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"week_start_dt"},"Name":"Dates.week_start_dt","NativeReferenceName":"Episode Week"},{"Measure":{"Expression":{"SourceRef":{"Source":"_"}},"Property":"epi_curve_text_color"},"Name":"_measures.epi_curve_text_color"},{"Measure":{"Expression":{"SourceRef":{"Source":"_"}},"Property":"epi_curve_text_avg_color"},"Name":"_measures.epi_curve_text_avg_color"},{"Measure":{"Expression":{"SourceRef":{"Source":"_1"}},"Property":"epi_curve_title"},"Name":"_titles.epi_curve_title"}],"Where":[{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"p"}},"Property":"Fields"}}],"Values":[[{"Literal":{"Value":"'''Dates''[week_start_dt]'"}}]]}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3,4,5],"Subtotal":1}]},"Projections":[6],"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"SuppressedJoinPredicates":[4,5],"Version":1},"ExecutionMetricsKind":1}}]},"CacheKey":"{\"Commands\":[{\"SemanticQueryDataShapeCommand\":{\"Query\":{\"Version\":2,\"From\":[{\"Name\":\"_\",\"Entity\":\"_measures\",\"Type\":0},{\"Name\":\"r\",\"Entity\":\"RollingAvgs\",\"Type\":0},{\"Name\":\"d\",\"Entity\":\"Dates\",\"Type\":0},{\"Name\":\"_1\",\"Entity\":\"_titles\",\"Type\":0},{\"Name\":\"p\",\"Entity\":\"Param - MWD\",\"Type\":0}],\"Select\":[{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"_\"}},\"Property\":\"n\"},\"Name\":\"_measures.n\",\"NativeReferenceName\":\"Case Count\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"_\"}},\"Property\":\"epi_curve_note\"},\"Name\":\"_measures.epi_curve_note\",\"NativeReferenceName\":\"Note\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"r\"}},\"Property\":\"rollingavg_table\"},\"Name\":\"RollingAvgs.rollingavg_table\",\"NativeReferenceName\":\"7-Day Average\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"d\"}},\"Property\":\"week_start_dt\"},\"Name\":\"Dates.week_start_dt\",\"NativeReferenceName\":\"Episode Week\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"_\"}},\"Property\":\"epi_curve_text_color\"},\"Name\":\"_measures.epi_curve_text_color\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"_\"}},\"Property\":\"epi_curve_text_avg_color\"},\"Name\":\"_measures.epi_curve_text_avg_color\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"_1\"}},\"Property\":\"epi_curve_title\"},\"Name\":\"_titles.epi_curve_title\"}],\"Where\":[{\"Condition\":{\"In\":{\"Expressions\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"p\"}},\"Property\":\"Fields\"}}],\"Values\":[[{\"Literal\":{\"Value\":\"'''Dates''[week_start_dt]'\"}}]]}}}]},\"Binding\":{\"Primary\":{\"Groupings\":[{\"Projections\":[0,1,2,3,4,5],\"Subtotal\":1}]},\"Projections\":[6],\"DataReduction\":{\"DataVolume\":3,\"Primary\":{\"Window\":{\"Count\":500}}},\"SuppressedJoinPredicates\":[4,5],\"Version\":1},\"ExecutionMetricsKind\":1}}]}","QueryId":"","ApplicationContext":{"DatasetId":"ef8eaa57-8505-48f9-98db-97efa77f32b3","Sources":[{"ReportId":"b2460244-f012-444e-9c95-81d48968da32","VisualId":"11bd57ce0058a8dc517c"}]}}],"cancelQueries":[],"modelId":797078}

In [2]:
response = requests.post(
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata?synchronous=true",
    headers=headers,
    json=payload
)
data = response.json()

In [3]:
rows = (
    data["results"][0]["result"]["data"]["dsr"]["DS"][0]["PH"][1]["DM1"]
)

# Extract records
records = []
for row in rows:
    cols = row["C"]
    # Some rows might have only a date or a note at the end
    if len(cols) >= 2:
        ts = cols[0]
        count = cols[1]
        # Convert the ms timestamp to date
        date = datetime.utcfromtimestamp(ts / 1000).date()
        records.append({"week_start_date": date, "cases": count})
    elif len(cols) == 1 and isinstance(cols[0], int):
        # Only date, count is missing
        date = datetime.utcfromtimestamp(cols[0] / 1000).date()
        records.append({"week_start_date": date, "cases": None})

# Save to CSV
df = pd.DataFrame(records)
df.to_csv("mpox_cases_by_week.csv", index=False)
print(df.head())

  week_start_date  cases
0      2022-05-15    1.0
1      2022-05-22    0.0
2      2022-05-29    4.0
3      2022-06-05    8.0
4      2022-06-12   18.0


/var/folders/t2/t39bprkn16dg9nr4v6c18v0w0000gn/T/ipykernel_46116/916753629.py:14: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(ts / 1000).date()
/var/folders/t2/t39bprkn16dg9nr4v6c18v0w0000gn/T/ipykernel_46116/916753629.py:18: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(cols[0] / 1000).date()
